# Comparison APIs on GPU Environment

## The Coding Environment

For your work today, you have access to several computational resources in the cloud. Run the following cell to see the features available to you today.

In [1]:
!nvidia-smi

Thu Jul 11 16:24:49 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P6000        On   | 00000000:3B:00.0 Off |                  Off |
| 26%   26C    P8     9W / 250W |     48MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!nvidia-smi topo -m 

While your work today will be on a single node, all the techniques you learn today, can be used to run your applications across clusters of multi-GPU nodes.

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Fri_Sep__8_19:17:24_PDT_2023
Cuda compilation tools, release 12.3, V12.3.52
Build cuda_12.3.r12.3/compiler.33281558_0


In [3]:
!pgcc --version


pgcc (aka nvc) 23.11-0 64-bit target on Linux -tp skylake-avx512 
PGI Compilers and Tools
Copyright (c) 2023, NVIDIA CORPORATION & AFFILIATES.  All rights reserved.

pgcc (aka nvc) 23.11-0 64-bit target on x86-64 Linux -tp skylake-avx512 
PGI Compilers and Tools
Copyright (c) 2023, NVIDIA CORPORATION & AFFILIATES.  All rights reserved.


In [4]:
!pgfortran --version


pgfortran (aka nvfortran) 23.11-0 64-bit target on x86-64 Linux -tp skylake-avx512 
PGI Compilers and Tools
Copyright (c) 2023, NVIDIA CORPORATION & AFFILIATES.  All rights reserved.


In [5]:
!clang --version

/bin/bash: clang: comando não encontrado


## Environment Modules on AIRIS

These modules must be initialized before running the jupyter-notebook:
```cpp
Currently Loaded Modulefiles:
    1) anaconda3/2023.07     
    2) ucx/1.15.0
    3) openmpi/4.1.5  
    4) nvhpc/23.11
    5) llvm/12.0.0
```

In [ ]:
!module load anaconda3/2023.07 ucx/1.15.0 openmpi/4.1.5 nvhpc/23.11

## `Matrix Multiple Benchmarks`

### ⊗ Sequential

In [6]:
%%writefile mm.c
#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>

void fill_matrix(double *A, int n){

  for(int i = 0; i < n; i++)
    for(int j = 0; j < n; j++)
      A[i*n+j] = rand()%(10-1)*1;

}

void print_matrix(double *A, int n){

  int i,j;

  for(int i = 0; i < n; i++){
    for(int j = 0; j < n; j++)
      printf("%1.2f\t", A[i*n+j]);
    printf("\n");
  }

  printf("\n");

}

int main(int argc, char **argv){

 int n = atoi(argv[1]);
 int i, j, k;

 double  *A = (double *) malloc (sizeof(double) * n * n);
 double  *B = (double *) malloc (sizeof(double) * n * n);
 double  *C = (double *) malloc (sizeof(double) * n * n);

 fill_matrix(A,n);
 fill_matrix(B,n);

 //Measuring time
  struct timeval begin, end;
  gettimeofday(&begin, 0);

 for(i = 0; i < n; i++)
  for(j = 0; j < n; j++)
    for(k = 0; k < n; k++)
      C[i*n+j]+=A[i*n+k]*B[k*n+j];

 gettimeofday(&end, 0);
 double elapsed = (end.tv_sec - begin.tv_sec) + (end.tv_usec - begin.tv_usec)*1e-6;

 printf("%d x %d  %.3f seconds\n", n, n, elapsed);

 //print_matrix(A,n);
 //print_matrix(B,n);
 //print_matrix(C,n);

 return 0;
}

Writing mm.c


In [7]:
!gcc mm.c -o mm -O3

In [8]:
!./mm 1000

1000 x 1000  1.244 seconds


### ⊗ OpenMP

In [9]:
%%writefile mm-omp.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>
#include <sys/time.h>

void fill_matrix(double *A, int n)
{
  for(int i = 0; i < n; i++)
    for(int j = 0; j < n; j++)
      A[i*n+j] = rand()%(10-1)*1;
}

void print_matrix(double *A, int n)
{
  for(int i = 0; i < n; i++){
    for(int j = 0; j < n; j++)
      printf("%1.2f\t", A[i*n+j]);
   printf("\n");
  }

  printf("\n");
}

int main(int argc, char **argv)
{
  int n = atoi(argv[1]);  
  int i, j, k;
  struct timeval begin, end;
  
  double  *A = (double *) malloc(sizeof(double) * n * n);
  double  *B = (double *) malloc(sizeof(double) * n * n);
  double  *C = (double *) malloc(sizeof(double) * n * n);

  fill_matrix(A,n);
  fill_matrix(B,n);

  gettimeofday(&begin, 0);
     
  #pragma omp parallel for private(i,j,k)
   for(i = 0; i < n; i++) 
    for(j = 0; j < n; j++)
      for(k = 0; k < n; k++) 
        C[i*n+j] += A[i*n+k] * B[k*n+j];
    
   gettimeofday(&end, 0);
  
   long seconds = end.tv_sec - begin.tv_sec;
   long microseconds = end.tv_usec - begin.tv_usec;
   double elapsed = seconds + microseconds*1e-6;
    
   printf("%d x %d  %.2f seconds\n", n, n, elapsed);    
    
   //print_matrix(A,n);
   //print_matrix(B,n);
   //print_matrix(C,n);

   return 0;
}

Writing mm-omp.c


In [10]:
!gcc mm-omp.c -o mm-omp -fopenmp -O3

In [11]:
!OMP_NUM_THREADS=32 ./mm-omp 1000

1000 x 1000  0.06 seconds


### ⊗ CUDA

In [12]:
%%writefile mm-CUDA.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <sys/time.h>

__global__ void kernel(double *A, double *B, double *C, int n) 
{  
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int j = blockIdx.y * blockDim.y + threadIdx.y;

  if(i < n && j < n)
    for( int k = 0; k < n; k++) 
       C[i*n+j] += A[i*n+k] * B[k*n+j];

}
 
void mult_matrix_cpu(double *A, double *B, double *C, int n) 
{
   for(int i = 0; i < n; i++) 
      for(int j = 0; j < n; j++)
         for(int k = 0; k < n; k++) 
            C[i*n+j]+=A[i*n+k]*B[k*n+j];
        
}

void fill_matrix(double *A, int n)
{ 
   for(int i=0; i < n; i++)
     for(int j=0; j < n; j++)
       A[i*n+j] = rand()%(10-1)*1;
   
}

void print_matrix(double *A, int n)
{
  for(int i = 0; i < n; i++){
    for(int j = 0; j < n; j++)
      printf("%1.2f\t", A[i*n+j]);
    printf("\n");
  }

  printf("\n");

}

int main(int argc, char **argv)
{
    int n = atoi(argv[1]);
    int sizeblock = atoi(argv[2]);
    struct timeval begin, end;

    /*Host*/
    double *A_host=(double *) malloc (n * n * sizeof(double));
    double *B_host=(double *) malloc (n * n * sizeof(double));
    double *C_host=(double *) malloc (n * n * sizeof(double));
        
    fill_matrix(A_host,n);
    fill_matrix(B_host,n);
      
    //print_matrix(A_host,n);
    //print_matrix(B_host,n);

    gettimeofday(&begin, 0);
    
    /*Device*/
    double *A_device;
    double *B_device;
    double *C_device;

    cudaMalloc((void**)&A_device, n * n * sizeof(double) ); 
    cudaMalloc((void**)&B_device, n * n * sizeof(double) ); 
    cudaMalloc((void**)&C_device, n * n * sizeof(double) ); 

    cudaMemcpy(A_device, A_host, n * n * sizeof(double), cudaMemcpyHostToDevice ); 
    cudaMemcpy(B_device, B_host, n * n * sizeof(double), cudaMemcpyHostToDevice ); 

    /*Computational GRID: (Grid: 2D Block: 2D)*/
    dim3 NUMBER_OF_BLOCKS ( (int) ceil( (float) n / sizeblock), (int) ceil( (float)n / sizeblock) );
    dim3 NUMBER_OF_THREADS( sizeblock, sizeblock);  

          kernel<<< NUMBER_OF_BLOCKS, NUMBER_OF_THREADS >>>(A_device, B_device, C_device, n);      

    cudaMemcpy(C_host, C_device, n * n * sizeof(double), cudaMemcpyDeviceToHost ); 

    //print_matrix(C_host, n );

    gettimeofday(&end, 0);
    
    long seconds = end.tv_sec - begin.tv_sec;
    long microseconds = end.tv_usec - begin.tv_usec;
    double elapsed = seconds + microseconds*1e-6;
    
    printf("%d x %d  %.3f seconds\n", n, n, elapsed);  
    
    cudaFree(A_device );
    cudaFree(B_device );
    cudaFree(C_device );
  
    return 0;
}

Writing mm-CUDA.cu


In [13]:
!nvcc mm-CUDA.cu -o mm-CUDA

In [14]:
!./mm-CUDA 1000 64

1000 x 1000  0.181 seconds


### ⊗ OpenACC

#### C

In [15]:
%%writefile mm-openacc.c
#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>

void fill_matrix(double *A, int n)
{
  for(int i = 0; i < n; i++)
    for(int j = 0; j < n; j++)
      A[i*n+j] = rand()%(10-1)*1; 
}

void print_matrix(double *A, int n)
{
  for(int i = 0; i < n; i++){
    for(int j = 0; j < n; j++)
      printf("%1.2f\t", A[i*n+j]);
    printf("\n");
  }

  printf("\n");

}

int main(int argc, char **argv)
{
  int n = atoi(argv[1]);  
  int i, j, k;
  struct timeval begin, end;
 
  double *A = (double *) malloc (sizeof(double) * n * n);
  double *B = (double *) malloc (sizeof(double) * n * n);
  double *C = (double *) malloc (sizeof(double) * n * n);

  fill_matrix(A,n);
  fill_matrix(B,n);
 
  gettimeofday(&begin, 0);
      
  #pragma acc data present_or_copyin(A[:n*n], B[:n*n], n) copyout(C[:n*n])
   #pragma acc parallel 
    #pragma acc loop
     for(i = 0; i < n; i++) 
       for(j = 0; j < n; j++)
         for(k = 0; k < n; k++) 
           C[i*n+j] += A[i*n+k] * B[k*n+j];

   gettimeofday(&end, 0); 
  
   long seconds = end.tv_sec - begin.tv_sec;
   long microseconds = end.tv_usec - begin.tv_usec;
   double elapsed = seconds + microseconds*1e-6;
    
    printf("%d x %d  %.2f seconds\n", n, n, elapsed);  
     
  //print_matrix(A,n);
  //print_matrix(B,n);
  //print_matrix(C,n);

  return 0;
}

Writing mm-openacc.c


In [16]:
!pgcc mm-openacc.c -o mm-openacc -acc

In [17]:
!./mm-openacc 1000

1000 x 1000  0.47 seconds


#### Fortran

In [18]:
%%writefile mm-fortran.f90
program matrix_multiply
  use openacc
  implicit none

  integer :: n, i, j, k
  real, allocatable :: A(:,:), B(:,:), C(:,:)
  character(len=100) :: arg
    
  integer :: start_count, end_count, rate, count_max
  real :: elapsed_time  

  ! Get the command-line argument for the matrix size
  call get_command_argument(1, arg)
  read(arg, *) n

  ! Allocate matrices
  allocate(A(n, n))
  allocate(B(n, n))
  allocate(C(n, n))

  ! Initialize matrices A and B
  call random_number(A)
  call random_number(B)
  C = 0.0

  ! Get the clock rate (ticks per second) and the maximum count value
  call system_clock(count_max=count_max, count_rate=rate)

  ! Get the start time
  call system_clock(start_count)

  ! Matrix multiplication using OpenACC
  !$acc data copyin(A, B), copyout(C)
  !$acc parallel loop collapse(2)
  do i = 1, n
     do j = 1, n
        do k = 1, n
           C(i, j) = C(i, j) + A(i, k) * B(k, j)
        end do
     end do
  end do
  !$acc end parallel loop
  !$acc end data

  ! Get the end time
  call system_clock(end_count)

  ! Calculate the elapsed time in seconds
  elapsed_time = real(end_count - start_count) / rate

  ! Print the elapsed time
  print '(I0, " x ", I0, "  ", F0.2, " seconds")', n, n, elapsed_time

  ! Deallocate matrices
  deallocate(A)
  deallocate(B)
  deallocate(C)

end program matrix_multiply

Writing mm-fortran.f90


In [19]:
!pgfortran mm-fortran.f90 -o mm-fortran -acc  

In [20]:
!./mm-fortran 1000

1000 x 1000  0.12 seconds


### ⊗ OpenMP5

In [21]:
%%writefile mm-omp5.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>
#include <sys/time.h>

void fill_matrix(double *A, int n)
{ 
  for(int i = 0; i < n; i++)
    for(int j = 0; j < n; j++)
      A[i*n+j] = rand()%(10-1)*1; 
}

void print_matrix(double *A, int n)
{
  for(int i = 0; i < n; i++){
    for(int j = 0; j < n; j++)
      printf("%1.2f\t", A[i*n+j]);
    printf("\n");
  }
  
  printf("\n");
}

int main(int argc, char **argv)
{
  int n = atoi(argv[1]);  
  int i, j, k;
  struct timeval begin, end;

  double  *A = (double *) malloc (sizeof(double) * n * n);
  double  *B = (double *) malloc (sizeof(double) * n * n);
  double  *C = (double *) malloc (sizeof(double) * n * n);

  fill_matrix(A,n);
  fill_matrix(B,n);

  gettimeofday(&begin, 0);
    
  #pragma omp target data map(to:A[:n*n], B[:n*n], n) map(from:C[:n*n])
  {
   #pragma omp target teams distribute parallel for private(i,j,k)
   for(i = 0; i < n; i++) 
     for(j = 0; j < n; j++)
       for(k = 0; k < n; k++) 
         C[i*n+j] += A[i*n+k] * B[k*n+j];
  }

   gettimeofday(&end, 0); 
    
   long seconds = end.tv_sec - begin.tv_sec;
   long microseconds = end.tv_usec - begin.tv_usec;
   double elapsed = seconds + microseconds*1e-6;
    
    printf("%d x %d  %.2f seconds\n", n, n, elapsed);   
  
  //print_matrix(A,n);
  //print_matrix(B,n);
  //print_matrix(C,n);

  return 0;
}

Writing mm-omp5.c


In [ ]:
!clang mm-omp5.c -o mm-omp5 -fopenmp -fopenmp-targets=nvptx64-nvidia-cuda

In [ ]:
!./mm-omp5 1000

### Table with Comparison Performance Analysis using 1-GPU

| Program Version      | Execution Time (sec.)  | Speedup      |
| :---                 |    :----:              |        ---:  |
| Serial               | 1.50                   | 1X           |
| OpenMP T=36          | 0.06                   | 25X          |
| OpenACC              | 0.57                   | 2.63X        | 
| CUDA                 | 0.20                   | 7.5X         | 
| OpenMP5              | 2.24                   | -            | 

## Limpando os arquivos remanescentes

In [ ]:
!rm -rf mm*